## Libraries

In [86]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate, Activation, Concatenate
from tensorflow.keras.metrics import IoU, BinaryIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
import cv2 as cv

## Loading Data

In [68]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1) / 255 
    return image, mask

In [69]:
def batch_data (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size)

In [120]:
home = os.environ['HOME']
path_X = os.path.join(home,'raw_data/small_dataset/sample_images/')
path_y = os.path.join(home,'raw_data/small_dataset/sample_masks/')
dataset = batch_data(path_X=path_X, path_y=path_y, batch_size=32)

## Model Definition

In [130]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [131]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters) #can be used as skip connection 
    p = MaxPooling2D((2,2))(x)
    return x, p

In [132]:
def decoder_block(inputs, skip_features, num_filters): #skip features are going to be the x returned from the encoder block
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [136]:
def build_unet(img_height, img_width, channels):
    
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder - downscaling (creating features)
    skip1, pool1 = encoder_block(inputs, 16)
    skip2, pool2 = encoder_block(pool1, 32) 
    skip3, pool3 = encoder_block(pool2, 64)
    skip4, pool4 = encoder_block(pool3, 128) 
    
    #Bottleneck or bridge between encoder and decoder
    b1 = conv_block(pool4, 256)
    
    #Decoder - upscaling (reconstructing the image and giving it precise spatial location)
    decoder1 = decoder_block(b1, skip4, 128)
    decoder2 = decoder_block(decoder1, skip3, 64)
    decoder3 = decoder_block(decoder2, skip2, 32)
    decoder4 = decoder_block(decoder3, skip1, 16)
    
    #Output
    outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(decoder4)
    model = Model(inputs, outputs)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    return model


In [137]:
model = build_unet(256, 256, 3)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_259 (Conv2D)            (None, 256, 256, 16  448         ['input_29[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_234 (Batch  (None, 256, 256, 16  64         ['conv2d_259[1][0]']             
 Normalization)                 )                                                           

In [108]:
checkpoint_filepath = '../tmp/checkpoint'
es = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', restore_best_weights=True)

In [118]:
path_X

'/home/jupyter/raw_data/small_dataset/sample_images'

In [138]:
model.fit(dataset, epochs = 50, callbacks=[es], verbose=1)

Epoch 1/50


2022-12-01 05:26:37.757597: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 128.00MiB (rounded to 134217728)requested by op model_5/conv2d_259/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-12-01 05:26:37.757678: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-12-01 05:26:37.757692: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 843, Chunks in use: 842. 210.8KiB allocated for chunks. 210.5KiB in use in bin. 105.4KiB client-requested in use in bin.
2022-12-01 05:26:37.757700: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 242, Chunks in use: 242. 121.8KiB allocated for chunks. 121.8KiB in use in bin. 121.0KiB client-requested in use in bin.


ResourceExhaustedError: Graph execution error:

Detected at node 'model_5/conv2d_259/Conv2D' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.7/asyncio/bas

2022-12-01 05:26:37.795223: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 25 Chunks of size 9216 totalling 225.0KiB
2022-12-01 05:26:37.795229: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 10240 totalling 10.0KiB
2022-12-01 05:26:37.795235: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 11008 totalling 10.8KiB
2022-12-01 05:26:37.795241: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 11520 totalling 11.2KiB
2022-12-01 05:26:37.795246: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 13056 totalling 12.8KiB
2022-12-01 05:26:37.795253: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 25 Chunks of size 18432 totalling 450.0KiB
2022-12-01 05:26:37.795263: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 27648 totalling 27.0KiB
2022-12-01 05:26:37.795269: I tensorflow/core/common_runtime/bfc_allocator.cc:1074] 1 Chunks of size 28672 totalling 28.0K